In [1]:
#### run imports
%run "Initialize.ipynb"
matplotlib.rc('font', size=20)                   # Use big fonts...
plt.rcParams['figure.figsize'] = (10.0, 10.0)    # ... and big plots
#hax.make_minitrees=False #I just don't feel like making minitrees for now
### Perceptually uniform, good for greyscale and dichromacy: magma, plasma, inferno and viridis
### ROOTish: jet

plt.set_cmap('viridis')

In [8]:
# load data
# Get pax values
from pax import units, configuration, datastructure
pax_config = configuration.load_configuration('XENON1T')
n_channels = pax_config['DEFAULT']['n_channels']
pmts = pax_config['DEFAULT']['pmts']
tpc_height = pax_config['DEFAULT']['tpc_length']
tpc_radius = pax_config['DEFAULT']['tpc_radius']
gains = pax_config['DEFAULT']['gains']
# get channel number of busy on
busy_on_ch = pax_config['DEFAULT']['channels_in_detector']['busy_on'][0]



# hax configuration
import hax
from hax import cuts

hax.__version__
hax.init(experiment='XENON1T',
         raw_data_access_mode='local',
         raw_data_local_path='/project/lgrandi/xenon1t/raw',
        main_data_paths=['/project/lgrandi/xenon1t/processed/pax_v6.6.5' , 
                         '/project2/lgrandi/xenon1t/processed/pax_v6.6.5'], # add here correct path for AmBe
        #minitree_paths = ['/project*/lgrandi/xenon1t/minitrees', 
         #                 '/project*/lgrandi/xenon1t/minitrees/latest',
          #               '.', '/project*/lgrandi/xenon1t/minitrees/pax_v6.6.5'],
        #minitree_paths = ['/home/fieguth/DEC','*'],
         pax_version_policy= '6.6.5'  )
        #minitree_paths = ['/project*/lgrandi/xenon1t/minitrees/latest'],
         #pax_version_policy= '6.6.5'  )



datasets = hax.runs.datasets # this variable holds all dataset info

#treemaker for cs2_bottom from Adam Brown
from s2_top_bottom_new import S2TopBottom


In [9]:
## Take all background datasets which possibly contain 164 keV
#dsets = datasets[(datasets.source__type == 'none')| (datasets.source__type == 'None')]
dsets = hax.runs.tags_selection(include=['blinded'], exclude=['bad', 'messy', '*trip', '*quake','test'])
dsets = dsets[(dsets.source__type == 'none')| (dsets.source__type == 'None')] # 
dsets = dsets[(dsets.location != '')]
dsets['run_time_s'] = pd.to_timedelta(pd.to_datetime(dsets.end) - pd.to_datetime(dsets.start)).dt.seconds
dsets = dsets[dsets.run_time_s > 3600] # select only runs of an hour or more, CHECK IF NECESSARY FOR AMBE
# some of the datasets had anomalously high background rates. I didn't use these datasets for the analysis.
# dsets['background_rate'] = dsets.trigger__events_built/dsets.run_time_s # background rate (avg. events/s)

# select data range
dsets = dsets[(dsets.start > pd.to_datetime('02/01/2017')) & (dsets.end < pd.to_datetime('06/05/2017'))]
#dsets = dsets[(dsets.start > pd.to_datetime('1/10/2017')) & (dsets.end < pd.to_datetime('1/18/2017'))]
dsets['start_date'] = dsets.start.dt.date

run_names = dsets.name

# get rid of problematic runs
bad_runs = ['161204_1517','170111_031','170111_0314'] # n'o minitree'

for run in bad_runs:
    run_names = run_names[run_names != run]
    dsets  = dsets[dsets.name != run]

In [ ]:
# load data or create minitrees from data
# a preselection for the activated lines is done by removing neutrons in AmBe and Kr83m in the respective data, additionally gas events are removed and z is cut for the final fiducial volume
preselect=['z>-81.9' ,'z<-20','x<47','y<47','x>-47','y>-47','s2_area_fraction_top < 0.74', 'cs2>0', 'cs1>0']
#df = hax.minitrees.load(run_names, ['Basics',S2TopBottom,'Extended'], preselection=preselect, force_reload=False, num_workers = 4)
df = hax.minitrees.load(run_names, ['Basics','Extended'], preselection=preselect, force_reload=True, num_workers = 5)
#hax.minitrees.save_cache_file(df,'cached_minitrees.mts')

Run 170202_1747: Making Extended minitree: 100%|██████████| 21603/21603 [01:00<00:00, 356.75it/s] 
/project/lgrandi/anaconda3/envs/pax_head/lib/python3.4/site-packages/pandas/computation/align.py:98: RuntimeWarning: divide by zero encountered in log10
  ordm = np.log10(abs(reindexer_size - term_axis_size))
Run 170420_2111: Making Extended minitree:  53%|█████▎    | 10327/19423 [00:37<01:02, 146.38it/s]

In [21]:
#df = hax.minitrees.load(run_names, ['Basics', 'TotalProperties', 'Fundamentals'], 
#                        cache_file = 'some_file_name')
#df = hax.minitrees.load_cache_file('some_file_name')

#df.head(10)

In [22]:
# Jelle provided me this to make wiki tables directly, big thanks!

def dataframe_to_wiki(df, float_digits=5, title='Awesome table'):
    table = '^ %s ' % title + '^' * (len(df.columns) - 1) + '^\n'
    table += '^ ' + ' ^ '.join(df.columns) + ' ^\n'
    def do_round(x):
        if isinstance(x, float):
            return round(x, float_digits)
        return x

    for _, row in df.iterrows():
        table += "| " + ' | '.join([str(do_round(x)) for x in row.values.tolist()]) + ' |\n'
    return table

print(dataframe_to_wiki(cuts.history(df), title='Table : Preselection overview'))

NameError: name 'df' is not defined

In [ ]:
# organize the data by day - taken from LOW-ER, but still useful
df = pd.merge(df, dsets[['number', 'start_date']], how = 'left', 
              left_on = 'run_number', right_on = 'number')

# list of dates of runs
dates = df.start_date.unique()
 
#obtain the total runtime
run_time_tot_s = dsets.run_time_s.sum()
print('Dataset metadata:')
print('%i datasets are used in this analysis.' % len(run_names))
print('The total runtime (sum of all the run times) is about %.3g hours.' % (run_time_tot_s/(60*60)))
print('There are %.f events loaded' % len(df))

In [ ]:
df['cs2bottom']=df.cs2_bottom_new
df['r_squared'] = df.x**2 + df.y**2  # Define a radial coordinate
df['r'] = np.sqrt(df.x**2 + df.y**2)
df['cs1cs2blog10'] = np.log10(df['cs2bottom']/df['cs1']) # define log-variable for useful plots
print(tpc_height)
# fiducial volume variables
z_min         = - 83.45 # cm 53.45
z_max         = - 13.45 # cm
r_max = 39.5

r_squared_max = r_max**2 # cm^2
r_tpc = (tpc_radius-5)**2
rho_xenon     = 3.1  # grams per cubic centimeter
fid_vol       = np.pi * r_squared_max * (z_max - z_min) # in units of cubic centimeters
n_kg          = fid_vol * rho_xenon / 1000
exposure = n_kg * run_time_tot_s / (60*60*24)

print('%g kg of xenon are within the fiducial volume.' % n_kg)

# some variable limits for further use
cs1_min       = 1 # pe for starters
cs2_min       = 150 # pe minimum trigger
cs1_max       = 31000 #pe resulting from alphas
cs2_max       = 1e7 #pe random high number so far
ces_min       = 0 # keV
ces_max       = 700 # keV

In [ ]:
################################### #### All cuts are here ############ ################################### 
from lax.lichens import sciencerun0 
df_work = df 
cut = sciencerun0.FiducialCylinder1T() 
cut2 = sciencerun0.S2AreaFractionTop() 
cut3= sciencerun0.S1SingleScatter()
cut4= sciencerun0.S2SingleScatter()

df_work = cut.process(df_work) 
df_work = cut2.process(df_work) 
df_work = cut3.process(df_work) 
df_work = cut4.process(df_work) 

#Cuts 

df_work = cuts.selection(df_work, df_work['CutFiducialCylinder1T'], desc='CutFiducialCylinder1T')
df_work = cuts.selection(df_work, df_work['CutS2AreaFractionTop'], desc='CutS2AreaFractionTop') 
#df_work =cuts.selection(df_work,df_work.largest_other_s2<0.03*df_work.s2,'S2 single scatter cut')
df_work = cuts.selection(df_work, df_work['CutS2SingleScatter'], desc='CutS2SingleScatter') 
df_work = cuts.selection(df_work, df_work['CutS1SingleScatter'], desc='CutS1SingleScatter') 

cuts.history(df_work)

In [ ]:

plt.hist2d(df_work['cs1'], df_work['cs2bottom']/100, 
           bins=400, range=([0,17000],[0,15000]), norm=matplotlib.colors.LogNorm())
plt.colorbar(label='Number of events')
plt.xlabel('cS1 [pe]')
plt.ylabel('cS2$_b$ [pe]')


plt.show()

In [ ]:

plt.hist2d(np.sqrt(np.add(np.power(df_work['x'],2),np.power(df_work['y'],2))), df_work['z'], 
           bins=400, range=([0,50],[-100,0]), norm=matplotlib.colors.LogNorm())
plt.colorbar(label='Number of events')
plt.xlabel('cS1 [pe]')
plt.ylabel('cS2$_b$ [pe]')


plt.show()

## Activated Xe at 163.9 keV and 232.2 keV

In [ ]:
import ROOT
import root_numpy

ROOT.gROOT.SetStyle("Plain");   
ROOT.gStyle.SetOptStat(1000000010)
ROOT.gStyle.SetPalette(57);     
ROOT.gStyle.SetOptTitle(0);     
ROOT.gStyle.SetOptFit(111)
ROOT.TVirtualFitter.SetDefaultFitter("Minuit");

df_active = cuts.selection(df_work, df_work['CutFiducialCylinder1T'], desc='CutFiducialCylinder1T')
t164_236 = root_numpy.array2tree(df_active.to_records())
c = ROOT.TCanvas('c','c',900,600)

t164_236.Draw('cs2bottom/100:cs1>>h5(200,800,2800,70,200,900)','','COLZ')
hist_s2_s1_164_236kev = ROOT.gDirectory.Get('h5')
hist_s2_s1_164_236kev.GetXaxis().SetTitle('cs1')
hist_s2_s1_164_236kev.GetYaxis().SetTitle('cs2_b')
hist_s2_s1_164_236kev.Sumw2(1)

# Reposition the stats box
ROOT.gPad.Update()
st = hist_s2_s1_164_236kev.FindObject("stats")
st.SetX1NDC(0.625)
st.SetX2NDC(0.9)
st.SetY1NDC(0.275)
st.SetY2NDC(0.875)


#f7 = ROOT.TF2("f7","([0]*TMath::Exp(-0.5*(TMath::Power((((x-[1])*cos(-TMath::ATan([2]/[1])+TMath::Pi()/2)-(y-[2])*sin(-TMath::ATan([2]/[1])+TMath::Pi()/2))/[3]),2)+TMath::Power((((x-[1])*sin(-TMath::ATan([2]/[1])+TMath::Pi()/2)+(y-[2])*cos(-TMath::ATan([2]/[1])+TMath::Pi()/2))/[4]),2))))+([5]*TMath::Exp(-0.5*(TMath::Power((((x-[6])*cos(-TMath::ATan([7]/[6])+TMath::Pi()/2)-(y-[7])*sin(-TMath::ATan([7]/[6])+TMath::Pi()/2))/[8]),2)+TMath::Power((((x-[6])*sin(-TMath::ATan([7]/[6])+TMath::Pi()/2)+(y-[7])*cos(-TMath::ATan([7]/[6])+TMath::Pi()/2))/[9]),2))))",170,310,170,350)
#f7.SetParameter(0, 20)
#f7.SetParameter(1, 250)
#f7.SetParLimits(1,240,265)
#f7.SetParameter(2, 300)
#f7.SetParLimits(2,280,320)
#f7.SetParameter(3, 60)
#f7.SetParameter(4, 80)
#f7.SetParameter(5, amplitude_80kev)
#f7.FixParameter(6, mu_x_80kev)
#f7.FixParameter(7, mu_y_80kev)
#f7.FixParameter(8, a_80kev)
#f7.FixParameter(9, b_80kev)
#f7.SetLineColor(1)
#f7.SetLineWidth(2)
#f7.SetContour(15, ) #What kind of contours are drawn by default anyway?


f6 = ROOT.TF2("f6","([0]*TMath::Exp(-0.5*(TMath::Power((((x-[1])*cos([3])-(y-[2])*sin([3]))/[4]),2)+TMath::Power((((x-[1])*sin([3])+(y-[2])*cos([3]))/[5]),2))))+([6]*TMath::Exp(-0.5*(TMath::Power((((x-[7])*cos([9])-(y-[8])*sin([9]))/[10]),2)+TMath::Power((((x-[7])*sin([9])+(y-[8])*cos([9]))/[11]),2))))+[12]"
              ,700,2200,200,900)
f6.SetParameter(0, 20)
f6.SetParLimits(0,0,1000)
f6.SetParameter(1, 1221)
#f6.SetParLimits(1,240,265)
f6.SetParameter(2, 414.9)
f6.SetParLimits(2,350,450)
f6.SetParameter(3,0.847)
f6.SetParameter(3, 0.6)
f6.SetParLimits(3,0.5,1.0);
f6.SetParameter(4, 37)
f6.SetParameter(5, 90)
f6.SetParameter(6, 5)
f6.SetParLimits(6, 0,1000)
f6.SetParameter(7, 1730)
f6.SetParLimits(7,1600,1800)
f6.SetParameter(8, 608)
f6.SetParLimits(8,490,720)
f6.SetParameter(9, 0.6)
f6.SetParLimits(9, 0.5,0.85)
f6.SetParameter(10, 110)
f6.SetParameter(11, 33)
f6.SetLineColor(2)
f6.SetLineWidth(3)
f6.SetContour(5, ) #What kind of contours are drawn by default anyway?

f6.SetParName(0, "A_{163.9 keV}")
f6.SetParName(1, "#mu_{S1, 163.9 keV}")
f6.SetParName(2, "#mu_{S2, 163.9 keV}")
f6.SetParName(3, "#phi_{163.9 keV}")
f6.SetParName(4, "a_{163.9 keV}")
f6.SetParName(5, "b_{163.9 keV}")
f6.SetParName(6, "A_{236.2 keV}")
f6.SetParName(7, "#mu_{S1, 236.2 keV}")
f6.SetParName(8, "#mu_{S2, 236.2 keV}")
f6.SetParName(9, "#phi_{236.2 keV}")
f6.SetParName(10, "a_{236.2 keV}")
f6.SetParName(11, "b_{236.2 keV}")
f6.SetParName(12, "Background")

#ROOT.gPad.SetLogz()


fit = hist_s2_s1_164_236kev.Fit('f6', "LER")
hist_s2_s1_164_236kev.GetXaxis().SetTitle('cS1 [pe]')
hist_s2_s1_164_236kev.GetYaxis().SetTitle('cS2_{b}/100 [pe]')
hist_s2_s1_164_236kev.GetYaxis().SetTitleOffset(1.3)
hist_s2_s1_164_236kev.Draw("COLZ")



# Get the right errors with option "E". Here a mean error is calculated from the asymmetric errors MINOS returns.
# The asymmetric errors can be accessed by r.LowerError(<parameter>) and r.UpperError(<parameter>)
# Also observe that the "S" option is used which lets you access the full covariance matrix, etc. 
r = ROOT.TFitResultPtr()
r = hist_s2_s1_164_236kev.Fit('f6', "RLES")
mu_x_164kev_err=r.Error(1)
mu_y_164kev_err=r.Error(2)
phi_164kev_err=r.Error(3)
mu_x_236kev_err=r.Error(7)
mu_y_236kev_err=r.Error(8)
phi_236kev_err=r.Error(9)

mu_x_164kev=f6.GetParameter(1)
mu_y_164kev=f6.GetParameter(2)
phi_164kev=f6.GetParameter(3)

mu_x_236kev=f6.GetParameter(7)
mu_y_236kev=f6.GetParameter(8)
phi_236kev=f6.GetParameter(9)


#print("Light yield is", mu_x_164_236kev/164, "PE/keV")
#print("Charge yield is", mu_y_164_236kev/100, "PE/keV")
#print("phi = ", phi_164_236kev)
#m = ROOT.TMath.Tan(phi_164_236kev*3.141/1164)
#print("Slope of the main axis is", m)




c.Update()


c.SaveAs('s2_s1_164_236kev.png')
c.SaveAs('s2_s1_164_236kev.pdf')
from IPython.display import Image
Image(filename='s2_s1_164_236kev.png')

## $^{60}$Co at 1173.2 keV and 1332.5 keV

In [ ]:
import ROOT
import root_numpy

ROOT.gROOT.SetStyle("Plain");   
ROOT.gStyle.SetOptStat(1000000010)
ROOT.gStyle.SetPalette(57);     
ROOT.gStyle.SetOptTitle(0);     
ROOT.gStyle.SetOptFit(111)
ROOT.TVirtualFitter.SetDefaultFitter("Minuit");

df_cobalt=cuts.selection(df_work,df_work.largest_other_s2<0.01*df_work.s2,'Strict S2 single scatter cut')
df_cobalt=cuts.selection(df_cobalt,np.sqrt(np.add(np.power(df_cobalt.x,2),np.power(df_cobalt.y,2)))<35,'Radial cut')


t164_236 = root_numpy.array2tree(df_cobalt.to_records())

c = ROOT.TCanvas('c','c',900,600)

t164_236.Draw('cs2bottom/100:cs1>>h5(140,5000,9000,160,3000,7000)','','COLZ')
hist_s2_s1_1173_1332kev = ROOT.gDirectory.Get('h5')
hist_s2_s1_1173_1332kev.GetXaxis().SetTitle('cs1')
hist_s2_s1_1173_1332kev.GetYaxis().SetTitle('cs2_b')
hist_s2_s1_1173_1332kev.Sumw2(1)

# Reposition the stats box
ROOT.gPad.Update()
st = hist_s2_s1_1173_1332kev.FindObject("stats")
st.SetX1NDC(0.725)
st.SetX2NDC(0.9)
st.SetY1NDC(0.275)
st.SetY2NDC(0.875)


f6 = ROOT.TF2("f6","([0]*TMath::Exp(-0.5*(TMath::Power((((x-[1])*cos([3])-(y-[2])*sin([3]))/[4]),2)+TMath::Power((((x-[1])*sin([3])+(y-[2])*cos([3]))/[5]),2))))+([6]*TMath::Exp(-0.5*(TMath::Power((((x-[7])*cos([9])-(y-[8])*sin([9]))/[10]),2)+TMath::Power((((x-[7])*sin([9])+(y-[8])*cos([9]))/[11]),2))))+[12]"
              ,5700,8500,3500,6500)
f6.SetParameter(0, 30)
f6.SetParLimits(0,0,100)
f6.SetParameter(1, 6500)
f6.SetParLimits(1,5400,7600)
f6.SetParameter(2, 4500.9)
f6.SetParLimits(2,3500,5450)
f6.SetParameter(3,0.847)
f6.SetParameter(3, 0.5)
f6.SetParLimits(3,0.5,0.85);
f6.SetParameter(4, 600)
f6.SetParameter(5, 270)
f6.SetParameter(6, 20)
f6.SetParLimits(6, 0,100)
f6.SetParameter(7, 7500)
f6.SetParLimits(7,6000,8800)
f6.SetParameter(8, 5250)
f6.SetParLimits(8,4900,7200)
f6.SetParameter(9, 0.5)
f6.SetParLimits(9, 0.5,0.85)
f6.SetParameter(10, 600)
f6.SetParameter(11, 270)
f6.SetParameter(12, 1)
f6.SetParLimits(12, 0, 5)
f6.SetLineColor(2)
f6.SetLineWidth(3)
f6.SetContour(5, ) #What kind of contours are drawn by default anyway?

f6.SetParName(0, "A_{1173 keV}")
f6.SetParName(1, "#mu_{S1, 1173 keV}")
f6.SetParName(2, "#mu_{S2, 1173 keV}")
f6.SetParName(3, "#phi_{1173 keV}")
f6.SetParName(4, "a_{1173 keV}")
f6.SetParName(5, "b_{1173 keV}")
f6.SetParName(6, "A_{1332 keV}")
f6.SetParName(7, "#mu_{S1, 1332 keV}")
f6.SetParName(8, "#mu_{S2, 1332 keV}")
f6.SetParName(9, "#phi_{1332 keV}")
f6.SetParName(10, "a_{1332 keV}")
f6.SetParName(11, "b_{1332 keV}")
f6.SetParName(12, "Background")

#ROOT.gPad.SetLogz()


fit = hist_s2_s1_1173_1332kev.Fit('f6', "LER")
hist_s2_s1_1173_1332kev.GetXaxis().SetTitle('cS1 [pe]')
hist_s2_s1_1173_1332kev.GetYaxis().SetTitle('cS2_{b}/100 [pe]')
hist_s2_s1_1173_1332kev.GetYaxis().SetTitleOffset(1.3)
hist_s2_s1_1173_1332kev.Draw("COLZ")



# Get the right errors with option "E". Here a mean error is calculated from the asymmetric errors MINOS returns.
# The asymmetric errors can be accessed by r.LowerError(<parameter>) and r.UpperError(<parameter>)
# Also observe that the "S" option is used which lets you access the full covariance matrix, etc. 
r = ROOT.TFitResultPtr()
r = hist_s2_s1_1173_1332kev.Fit('f6', "RLES")
mu_x_1173kev_err=r.Error(1)
mu_y_1173kev_err=r.Error(2)
phi_1173kev_err=r.Error(3)
mu_x_1332kev_err=r.Error(7)
mu_y_1332kev_err=r.Error(8)
phi_1332kev_err=r.Error(9)

mu_x_1173kev=f6.GetParameter(1)
mu_y_1173kev=f6.GetParameter(2)
phi_1173kev=f6.GetParameter(3)

mu_x_1332kev=f6.GetParameter(7)
mu_y_1332kev=f6.GetParameter(8)
phi_1332kev=f6.GetParameter(9)





c.Update()


c.SaveAs('s2_s1_1173_1332kev.png')
c.SaveAs('s2_s1_1173_1332kev.pdf')
from IPython.display import Image
Image(filename='s2_s1_1173_1332kev.png')

## $^{83m}$Kr 41.5 keV

In [ ]:
import ROOT
import root_numpy

ROOT.gROOT.SetStyle("Plain");   
ROOT.gStyle.SetOptStat(1000000010)
ROOT.gStyle.SetPalette(57);     
ROOT.gStyle.SetOptTitle(0);     
ROOT.gStyle.SetOptFit(111)
ROOT.TVirtualFitter.SetDefaultFitter("Minuit");

df_active = cuts.selection(df_work, df_work['CutFiducialCylinder1T'], desc='CutFiducialCylinder1T')
t42 = root_numpy.array2tree(df_active.to_records())
c = ROOT.TCanvas('c','c',900,600)

t42.Draw('cs2bottom/100:cs1>>h5(73,200,450,50,30,130)','','COLZ')
hist_s2_s1_42kev = ROOT.gDirectory.Get('h5')
hist_s2_s1_42kev.GetXaxis().SetTitle('cs1')
hist_s2_s1_42kev.GetYaxis().SetTitle('cs2_b')
hist_s2_s1_42kev.Sumw2(1)

# Reposition the stats box
ROOT.gPad.Update()
st = hist_s2_s1_42kev.FindObject("stats")
st.SetX1NDC(0.725)
st.SetX2NDC(0.9)
st.SetY1NDC(0.475)
st.SetY2NDC(0.875)


f7 = ROOT.TF2("f7","([0]*TMath::Exp(-0.5*(TMath::Power((((x-[1])*cos([3])-(y-[2])*sin([3]))/[4]),2)+TMath::Power((((x-[1])*sin([3])+(y-[2])*cos([3]))/[5]),2))))+[6]"
              ,280,450,30,130)
f7.SetParameter(0, 20)
f7.SetParLimits(0,0,100)
f7.SetParameter(1, 350)
#f7.SetParLimits(1,240,265)
f7.SetParameter(2, 70)
#f7.SetParLimits(2,350,450)
f7.SetParameter(3, 0.4)
f7.SetParLimits(3,0.2,1.0);
f7.SetParameter(4, 27)
f7.SetParameter(5, 30)
f7.SetParameter(6, 1)
f7.SetParLimits(6,0,5)
f7.SetLineColor(2)
f7.SetLineWidth(3)
f7.SetContour(5, ) #What kind of contours are drawn by default anyway?

f7.SetParName(0, "A_{41.5 keV}")
f7.SetParName(1, "#mu_{S1, 41.5 keV}")
f7.SetParName(2, "#mu_{S2, 41.5 keV}")
f7.SetParName(3, "#phi_{41.5 keV}")
f7.SetParName(4, "a_{41.5 keV}")
f7.SetParName(5, "b_{41.5 keV}")
f7.SetParName(6, "Background")


#ROOT.gPad.SetLogz()


fit = hist_s2_s1_42kev.Fit('f7', "LER")
hist_s2_s1_42kev.GetXaxis().SetTitle('cS1 [pe]')
hist_s2_s1_42kev.GetYaxis().SetTitle('cS2_{b}/100 [pe]')
hist_s2_s1_42kev.GetYaxis().SetTitleOffset(1.3)
hist_s2_s1_42kev.Draw("COLZ")



# Get the right errors with option "E". Here a mean error is calculated from the asymmetric errors MINOS returns.
# The asymmetric errors can be accessed by r.LowerError(<parameter>) and r.UpperError(<parameter>)
# Also observe that the "S" option is used which lets you access the full covariance matrix, etc. 
r = ROOT.TFitResultPtr()
r = hist_s2_s1_42kev.Fit('f7', "RLES")
mu_x_42kev_err=r.Error(1)
mu_y_42kev_err=r.Error(2)
phi_42kev_err=r.Error(3)
#phi_80kev_err=r.Error(9)

#print("Error for cS1 mean =", mu_x_42kev_err)
#print( "Error for cS2bottom/100 mean =", mu_y_42kev_err)

# Draw the ellipsis main axis
#y1=-(xmin-mu_x)+mu_y
#y2=-(xmax-mu_x)+mu_y
#eline = ROOT.TLine(xmin,y1,xmax,y2)
#eline.SetLineWidth(2)
#eline.Draw("SAME")


#amplitude_42kev = f7.GetParameter(0)
mu_x_42kev=f7.GetParameter(1)
mu_y_42kev=f7.GetParameter(2)
#phi_42kev = f7.GetParameter(3)
#a_42kev = f7.GetParameter(4)
#b_42kev = f7.GetParameter(5)
#phi_80kev = f7.GetParameter(9)

#print("Light yield is", mu_x_42kev/164, "PE/keV")
#print("Charge yield is", mu_y_42kev/100, "PE/keV")
#print("phi = ", phi_42kev)
#m = ROOT.TMath.Tan(phi_42kev*3.141/1164)
#print("Slope of the main axis is", m)




c.Update()


c.SaveAs('s2_s1_42kev.png')
c.SaveAs('s2_s1_42kev.pdf')
from IPython.display import Image
Image(filename='s2_s1_42kev.png')

In [ ]:
def chisquare(function, functionparams, x, y, yerr):
    '''Function to calculate the chi2 of a fit with "function" that yields "functionparams". 
    Works with up to 7 fit parameters. Returns reduced chi2, chi2, ndof'''
    
    nparams=len(functionparams)
    if nparams==1:    
        summands=np.divide(np.power(np.subtract(y,function(x, functionparams[0])),2),np.power(yerr,2))
    elif nparams==2:
        summands=np.divide(np.power(np.subtract(y,function(x, functionparams[0], functionparams[1])),2),np.power(yerr,2))
    elif nparams==3:
        summands=np.divide(np.power(np.subtract(y,function(x, functionparams[0], functionparams[1], functionparams[2])),2),np.power(yerr,2))
    elif nparams==4:
        summands=np.divide(np.power(np.subtract(y,function(x, functionparams[0], functionparams[1], functionparams[2], functionparams[3])),2),np.power(yerr,2))
    elif nparams==5:
        summands=np.divide(np.power(np.subtract(y,function(x, functionparams[0], functionparams[1], functionparams[2], functionparams[3],functionparams[4])),2),np.power(yerr,2))
    elif nparams==6:
        summands=np.divide(np.power(np.subtract(y,function(x, functionparams[0], functionparams[1], functionparams[2], functionparams[3],functionparams[4],functionparams[5])),2),np.power(yerr,2))
    elif nparams==7:
        summands=np.divide(np.power(np.subtract(y,function(x, functionparams[0], functionparams[1], functionparams[2], functionparams[3],functionparams[4],functionparams[5],functionparams[6])),2),np.power(yerr,2))

        
    ndof=len(x)-len(functionparams)
    chi2=np.sum(summands)
    chi2ndof=np.divide(chi2,ndof)
    #print(summands)
    #print(chi2)
    #print(ndof)
    #print(chi2ndof)
    return(chi2ndof, chi2, ndof)

In [ ]:
# Quantities from fit with all points
s1=[mu_x_42kev,mu_x_164kev,mu_x_236kev, mu_x_1173kev, mu_x_1332kev]
s2=[mu_y_42kev,mu_y_164kev,mu_y_236kev, mu_y_1173kev, mu_y_1332kev]
s1errfit=[mu_x_42kev_err,mu_x_164kev_err,mu_x_236kev_err, mu_x_1173kev_err, mu_x_1332kev_err]
s2errfit=[mu_y_42kev_err,mu_y_164kev_err,mu_y_236kev_err, mu_y_1173kev_err, mu_y_1332kev_err]
#s1errsyst=[1,2,25.7,10,10,10,67]
#s2errsyst=[2,4,28.7,10,10,10,60]
energy=[41.5, 163.9, 236.2, 1173.2, 1332.5]
#s1err=np.sqrt(np.add(np.power(s1errfit,2),np.power(s1errsyst,2)))
#s2err=np.sqrt(np.add(np.power(s2errfit,2),np.power(s2errsyst,2)))



# Derived quantities
s1yield=np.divide(s1,energy)
s2yield=np.divide(np.multiply(1,s2),energy)
yerr=np.divide(np.multiply(1,s2errfit),energy)
xerr=np.divide(s1errfit,energy)

# Linear fit
def linear1(x,a,b):
    return a*x+b

initparams=[(-1.40275986, 6.08900108)]
linearfitparams, linearfitcovariance = curve_fit(linear1, s1yield, s2yield, initparams, yerr)

xvaluesforplot = np.arange(5, 9.5, 0.01)

plt.plot(xvaluesforplot, linear1(xvaluesforplot, linearfitparams[0], linearfitparams[1]), 
         label='$Q=-\\alpha \cdot L_y +(1/\\beta)$', color='red', linewidth=3, linestyle="--")

print(linearfitparams)
print(linearfitcovariance[0][0], linearfitcovariance[1][1])


plt.errorbar(s1yield,s2yield,yerr,xerr, marker='o', color='black', linestyle="", markersize=10, label= '$L_y$ and $Q$ from fits')
plt.ylabel('$Q_{ee}$ [pe/keV]')
plt.xlabel('$L_y$ [pe/keV]')


i=0
for linevalue in energy:
    xvalue=s1yield[i]
    yvalue=s2yield[i]
    if linevalue > 236 and linevalue < 1000:
        plt.text(xvalue-0.9, yvalue, '%5.1f keV' % linevalue)
    else:
        plt.text(xvalue+0.1, yvalue, '%5.1f keV' % linevalue)    
    i=i+1


leg = plt.legend(fancybox=True,loc='best',fontsize = 'medium')
leg.get_frame().set_alpha(0.5)     


ces_a = -linearfitparams[0]*(1/linearfitparams[1])
ces_a_err = np.sqrt((linearfitcovariance[0][0]/linearfitparams[1])**2+(linearfitparams[0]*linearfitcovariance[1][1]/(linearfitparams[1])**2)**2)
ces_b = 1/linearfitparams[1]
ces_b_err= linearfitcovariance[1][1]/(linearfitparams[1])**2

g1=(13.7*1e-3)/ces_a
g1_err=(13.7*1e-3*ces_a_err)/ces_a**2

g2=(13.7*1e-3*100)/ces_b #13.7 comes from an average of 73 quanta/keV (information carriers)
g2_err=(13.7*1e-3*100*ces_b_err)/ces_b**2


print("CES parameter a = %5.4f ± %5.4f" % (ces_a, ces_a_err))
print("CES parameter b = %5.4f ± %5.4f" % (ces_b, ces_b_err))
print("g1 = %5.4f ± %5.4f" % (g1, g1_err))
print("g2 = %5.4f ± %5.4f" % (g2, g2_err))
print("Chi2/NDOF is %5.4f" % chisquare(linear1, linearfitparams, s1yield, s2yield, yerr)[0])

plt.text(5.2,1.6, ("$\chi^2/ndf= %5.4f$" % chisquare(linear1, linearfitparams, s1yield, s2yield, yerr)[0]), fontsize=20)
plt.text(5.2,1.4, ("$g_1= (%5.4f \pm %5.4f)$ pe" % (g1, g1_err)), fontsize=20)
plt.text(5.2,1.2, ("$g_2= (%5.4f \pm %5.4f)$ pe" % (g2, g2_err)), fontsize=20)

#plt.text(5.2,2.15, ("$\\alpha= (%5.4f \pm %5.4f)$ keV/pe" % (ces_a, np.around(ces_a_err+0.005,2))), fontsize=20)
#plt.text(5.2,2.0, ("$\\beta= (%5.4f \pm %5.4f)$ keV/pe" % (ces_b, np.around(ces_b_err+0.005,2))), fontsize=20)

plt.title('Cross-check of $g_1$ and $g_2$ with PAX 6.6.5 for SR1')
plt.savefig('linces.pdf', transparent=True)
plt.show()

## Now build a quick CES and search for other lines in the data we can work with

In [ ]:
# lets calculate a rough CES 

w_value= 0.0137

g1 =  0.1471 # 6_4_2 0.9
g2 =  11.45 #

df_work['CES_tmp'] = w_value*(df_work.cs2bottom/g2 + df_work.cs1/g1)


In [ ]:
#df_work = cuts.selection(df_work, df_work['largest_other_s1'] < 0.1 * df_work['s1'], desc="multiscatter cut s1" ,force_repeat =True )# remove smoothly cuts with a second s2 reasonably high,.1 is experimentally at this point


In [ ]:
# CES of 40 keV using an exponentially modified Gaussian (from here many thanks to C.Wittweg for insight in his code))

file = ROOT.TFile ("dec1tsr0.root","RECREATE")

ROOT.gROOT.SetStyle("Plain");   
ROOT.gStyle.SetOptStat(1000000010)
ROOT.gStyle.SetPalette(57);     
ROOT.gStyle.SetOptTitle(0);     
ROOT.gStyle.SetOptFit(111)
ROOT.TVirtualFitter.SetDefaultFitter("Minuit"); 


t40b = root_numpy.array2tree(df_work.to_records())
c = ROOT.TCanvas('c','c',900,600)

t40b.Draw('CES_tmp>>h40kevs1(30,50,80)','','E1')

hist_s1_40kev = ROOT.gDirectory.Get('h40kevs1')
hist_s1_40kev.GetXaxis().SetTitle('ces [keV]')
hist_s1_40kev.GetYaxis().SetTitle('Counts')
hist_s1_40kev.Sumw2(1)

hist_s1_40kev.Write()

# Reposition the stats box
ROOT.gPad.Update()
st = hist_s1_40kev.FindObject("stats")
st.SetX1NDC(0.65)
st.SetX2NDC(0.95)
st.SetY1NDC(0.65)
st.SetY2NDC(0.95)

#sigma = 3, tau = 1, mu =2
#histfit= ROOT.TF1("histfit", "gaus(0)+gaus(3)",25,53)
#histfit.SetParameter(0, 50000)
#histfit.SetParLimits(0,0,50000)
#histfit.SetParameter(1, 31)
#histfit.SetParLimits(1,0,35)
#histfit.SetParameter(2,3)
#histfit.SetParLimits(2, 0,6)
#histfit.SetParameter(3, 260000)
#histfit.SetParameter(4, 38)
#histfit.SetParameter(5, 5)

histfit= ROOT.TF1("histfit", "[3]",50,80)
histfit.SetParameter(0, 3)
histfit.SetParameter(1, 64)
histfit.SetParLimits(1, 60,68)
histfit.SetParameter(2,4)
histfit.SetParameter(3, 15)



histfit.SetParName(0, "A_{41.5 keV}")
histfit.SetParName(1, "#mu_{41.5 keV}")
histfit.SetParName(2, "#sigma_{41.5 keV}")
histfit.SetParName(3, "bkg")


histfit.SetLineColor(2)

hist_s1_40kev.Fit("histfit","LRE")
histfit.Draw("same")

mu_x_40kev_1D=histfit.GetParameter(2)
mu_x_40kev_1D_err=histfit.GetParError(2)


c.Update()

c.SaveAs('s1_40kev.png')
from IPython.display import Image
Image(filename='s1_40kev.png') 

#c.Write()
#file.Close()

In [ ]:
# calculate a very rough limit MEI STYLE

f_K = 1 # hier erstmal nur für den KK
efficiency = 0.9 
alpha = 1.64 # fuer 90%
M = 1000 # target mass in kg
N_A = 6.023e23 # Avogadro-Konstante
abundance = 0.1 # easy, right?
A = 124 # easy, right?
#delta_T = 34.6/365. #livetime
delta_T = (run_time_tot_s/(60*60*24))/365
delta_E = 3.0 # resolution
b = 20.483  # background_rate
B = b * delta_E
mu_up = alpha * np.sqrt(B)



half_life = np.log(2)*efficiency*f_K*abundance*np.sqrt(delta_T)*(M*N_A/A) * (1/mu_up)



print(half_life)
